In [ ]:
#Dependencies
import pandas as pd
import os
from datetime import datetime # I think this one can probably get deleted
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import seaborn as sns
import scipy.stats as stats

%matplotlib inline
sns.set()

In [ ]:
# Import Data as DataFrames
fireData = pd.read_csv("Clean_Data/CA_WildFires.csv")
droughtData = pd.read_csv("Clean_Data/CA_Droughts.csv")

In [ ]:
#Melissa M Code

In [ ]:
#End Melissa M Code

In [ ]:
#Start Anika Code

# Data Analysis - California Wildfire Severity 2000-2020


In [ ]:
# End Anika Code

In [ ]:
#start Melissa K Code


In [ ]:
#End Melissa K Code

In [ ]:
# Start Liz Code

In [ ]:
#End Liz Code